In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pandas as pd

In [0]:
df_ne_pd = pd.read_csv('/dbfs/FileStore/data/NOTEEVENTS_csv.gz', compression='gzip',error_bad_lines=False)

/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
 has_raised = await self.run_ast_nodes(code_ast.body, cell_name,

In [0]:
# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [0]:
spar_df = pandas_to_spark(df_ne_pd)

In [0]:
spar_df.registerTempTable("noteevents")

spar_df.filter(spar_df.CATEGORY=="Discharge summary") \
    .registerTempTable("noteevents2")

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

In [0]:
# ne_struct = StructType([StructField("row_id", IntegerType(), True),
#                       StructField("subject_id", IntegerType(), True),
#                       StructField("hadm_id", IntegerType(), True),
#                       StructField("chartdate", DateType(), True),
#                       StructField("category", StringType(), True),
#                       StructField("description", StringType(), True),
#                       StructField("cgid", IntegerType(), True),
#                       StructField("iserror", IntegerType(), True),
#                       StructField("text", StringType(), True)])
# df_ne = spark.read.csv("./data/NOTEEVENTS-2.csv",
# # df_ne = spark.read.csv("./data/NOTEEVENTS-2sample.csv",
#                        header=True,
#                        schema=ne_struct)
# df_ne.registerTempTable("noteevents")
# df_ne.filter(df_ne.category=="Discharge summary") \
#     .registerTempTable("noteevents2")
    
# i want to cache noteevents, but it's too big

# many icd to one hadm_id
diag_struct = StructType([StructField("ROW_ID", IntegerType(), True),
                          StructField("SUBJECT_ID", IntegerType(), True),
                          StructField("HADM_ID", IntegerType(), True),
                          StructField("SEQ_NUM", IntegerType(), True),
                          StructField("ICD9_CODE", StringType(), True)])
df_diag_m = spark.read.csv("dbfs:/FileStore/data/DIAGNOSES_ICD_csv.gz",
                           header=True,
                           schema=diag_struct) \
            .selectExpr("ROW_ID as row_id", 
                        "SUBJECT_ID as subject_id",
                        "HADM_ID as hadm_id",
                        "SEQ_NUM as seq_num",
                        "ICD9_CODE as icd9_code")
df_diag_m.registerTempTable("diagnoses_icd_m")
df_diag_m.cache()

Out[7]: DataFrame[row_id: int, subject_id: int, hadm_id: int, seq_num: int, icd9_code: string]

First error received, not sure why but I was able to translate the code below here to the sql code used in CMD 9, the sql code might result in different results then this code originally resulted in

In [0]:
# one icd to one hadm_id (take the smallest seq number as primary)
diag_o_rdd = df_diag_m.rdd.sortBy(lambda x: (x.hadm_id, x.subject_id, x.seq_num)) \
    .groupBy(lambda x: x.hadm_id) \
    .mapValues(list) \
    .reduceByKey(lambda x, y: x if x.seq_num < y.seq_num else y) \
    .map(lambda x: x[1][0])   # py3 can't unpack tuple, need to upack manually
df_diag_o = spark.createDataFrame(diag_o_rdd)
df_diag_o.registerTempTable("diagnoses_icd_o")
df_diag_o.cache()


Out[8]: DataFrame[row_id: bigint, subject_id: bigint, hadm_id: bigint, seq_num: bigint, icd9_code: string]

In [0]:

# SQL Code start, replaced spark logic in command 8
# diagnoses_icd_o = spark.sql('''
# select distinct a.*
# from diagnoses_icd_m a
# inner join (
#    select distinct
#     subject_id as subject_id,
#     min(hadm_id) as hadm_id,
#     min(seq_num) as seq_num
#   from diagnoses_icd_m
#   group by 1
# ) b 
# on a.subject_id = b.subject_id
# and a.hadm_id = b.hadm_id
# and a.seq_num = b.seq_num
# ''')

# diagnoses_icd_o.registerTempTable("diagnoses_icd_o")
# diagnoses_icd_o.cache()
# SQL Code end, replaced spark logic in command 8

# original code worked correctley start

df_hadm_id_list = spark.sql("""
SELECT DISTINCT hadm_id FROM noteevents2
""")
df_hadm_id_list.registerTempTable("hadm_id_list")
df_hadm_id_list.cache()

# get subject_id list in noteevents
df_subject_id_list = spark.sql("""
SELECT DISTINCT subject_id FROM noteevents2
""")
df_subject_id_list.registerTempTable("subject_id_list")
df_subject_id_list.cache()

df_icd9desc = spark.read.csv("dbfs:/FileStore/data/D_ICD_DIAGNOSES_csv.gz",
                       header=True, inferSchema=True)
df_icd9desc.registerTempTable("diagnoses_icd_desc")

df_diag_o2 = spark.sql("""
SELECT row_id, subject_id, diagnoses_icd_o.hadm_id AS hadm_id,
seq_num, icd9_code
FROM diagnoses_icd_o JOIN hadm_id_list
ON diagnoses_icd_o.hadm_id = hadm_id_list.hadm_id
""")
df_diag_o2.registerTempTable("diagnoses_icd_o2")
df_diag_o2.cache()

df_diag_m2 = spark.sql("""
SELECT row_id, subject_id, diagnoses_icd_m.hadm_id AS hadm_id,
seq_num, icd9_code
FROM diagnoses_icd_m JOIN hadm_id_list
ON diagnoses_icd_m.hadm_id = hadm_id_list.hadm_id
""")
df_diag_m2.registerTempTable("diagnoses_icd_m2")
df_diag_m2.cache()
# original code worked correctley end

Out[9]: DataFrame[row_id: int, subject_id: int, hadm_id: int, seq_num: int, icd9_code: string]

In [0]:
# this block is mostly original code minus a minor tweak, and it works as expected
icd9code_score_hadm = spark.sql("""
SELECT icd9_code, COUNT(DISTINCT hadm_id) AS score
FROM diagnoses_icd_m2
GROUP BY icd9_code
""").rdd.cache()

icd9code_score_subj = spark.sql("""
SELECT icd9_code, COUNT(DISTINCT subject_id) AS score
FROM diagnoses_icd_m2
GROUP BY icd9_code
""").rdd.cache()

icd9cat_score_hadm = spark.sql("""
SELECT icd9_code, COUNT(DISTINCT hadm_id) AS score
FROM diagnoses_icd_m2
GROUP BY icd9_code
""").rdd.cache()

icd9cat_score_subj = spark.sql("""
SELECT icd9_code, COUNT(DISTINCT subject_id) AS score
FROM diagnoses_icd_m2
GROUP BY icd9_code
""").rdd.cache()

def get_id_to_topicd9(id_type, icdcode, topX):
    if id_type == "hadm_id" and icdcode:
        icd9_score = icd9code_score_hadm
    elif id_type == "hadm_id" and not icdcode:
        icd9_score = icd9cat_score_hadm
    elif id_type == "subject_id" and icdcode:
        icd9_score = icd9code_score_subj
    elif id_type == "subject_id" and not icdcode:
        icd9_score = icd9cat_score_subj
    else: #default
        icd9_score = icd9code_score_hadm
    
        
    icd9_topX2 = [i.icd9_code for i in icd9_score.takeOrdered(topX, key=lambda x: -x.score)]
    if not icdcode:
        icd9_topX2 = ['c'+str(i) for i in icd9_topX2]
    else:
        icd9_topX2 = [str(i) for i in icd9_topX2]
    icd9_topX = set(icd9_topX2)
    
    id_to_topicd9 = df_diag_m2.rdd \
        .map(lambda x: (x.hadm_id if id_type=="hadm_id" else x.subject_id, x.icd9_code if icdcode else 'c'+str(x.icd9_code))) \
        .groupByKey() \
        .mapValues(lambda x: set(x) & icd9_topX) \
        .filter(lambda t: t[0])
        
    return id_to_topicd9, list(icd9_topX2)

In [0]:

top_50_icd_hadm = spark.sql('''SELECT distinct icd9_code
,COUNT(DISTINCT hadm_id) AS score
FROM diagnoses_icd_m2
GROUP BY icd9_code
order by 2 desc
limit 50''')

top_50_icd_hadm.registerTempTable("top_50_icd_hadm")

hadm_top_50_icd_look_up = spark.sql('''
select distinct hadm_id  
,b.icd9_code
from diagnoses_icd_m2 a
left join top_50_icd_hadm b
''')

hadm_top_50_icd_look_up.registerTempTable("hadm_top_50_icd_look_up")

hadm_top_50_icd_step_0 = spark.sql('''
select distinct 
  a.hadm_id,
  a.icd9_code,
  sum(case when b.icd9_code is not null then 1 else 0 end) as present
from hadm_top_50_icd_look_up a
left join diagnoses_icd_m2 b
  on a.hadm_id = b.hadm_id
  and a.icd9_code = b.icd9_code
group by 1 , 2
''')

atl = hadm_top_50_icd_step_0.toPandas()
atl_0 = atl.pivot_table('present', ['hadm_id'], 'icd9_code').reset_index()


In [0]:
atl_1 = pandas_to_spark(atl_0)
atl_1.registerTempTable("top_50_icd_hadm_base")

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

In [0]:
%sql 
select count(1) 
from top_50_icd_hadm_base
limit 10

count(1)
52726


In [0]:
hadm_top_50_icd_step_1 = spark.sql('''
select distinct 
    a.*,
    b.row_id,
    b.text
from top_50_icd_hadm_base a 
inner join noteevents b
on a.hadm_id = b.hadm_id
''')
hadm_top_50_icd_step_1.registerTempTable("hadm_top_50_icd_step_1")

In [0]:
%sql
select * 
from hadm_top_50_icd_step_1
limit 10

hadm_id,0389,2449,25000,2720,2724,2760,2761,2762,2767,2851,2859,2875,3051,311,32723,4019,40390,40391,41071,412,41401,4168,4240,4241,42731,42789,4280,486,49390,496,5070,5119,5180,51881,53081,5845,5849,5859,5990,7742,78552,99592,9971,V053,V1582,V290,V4581,V4582,V5861,V5867,row_id,text
185973,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,159161,Normal sinus rhythm. Left atrial abnormality. Incomplete right bundle-branch block. Compared to the previous tracing of [**2200-4-22**] there is no significant change.
160055,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,159140,Tracing shows probable atrial flutter with 2:1 conduction. Probable inferior myocardial infarction of indeterminate age is possibly acute. Diffuse non-specific ST-T wave abnormalities. Compared to the previous tracing of [**2159-3-2**] atrial flutter is new. ST-T wave abnormalities are more marked and evidence for inferior injury is also more marked raising the possibility of an acute process. Repeat tracing and clinical correlation are suggested. TRACING #1
126989,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,159159,"Sinus rhythm Left axis deviation Left bundle branch block Tall T waves - consider acute ischemia or hyperkalemia Since previous tracing of [**2134-10-26**], peaked T waves are noted"
160055,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,159141,Sinus rhythm. Prolonged P-R interval. Non-specific ST-T wave changes. Compared to the previous tracing of [**2159-2-9**] the P-R interval has prolonged.
185973,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,159163,Atrial fibrillation with rapid ventricular response. Compared to the previous tracing the rate and rhythm have changed. TRACING #2
107791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,159155,Baseline artifact Sinus rhythm Atrial premature complex Otherwise probably normal ECG Since previous tracing of [**2158-4-2**]: atrial premature complex seen
185973,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,159160,"The tracing is marred by baseline artifact, precluding adequate interpretation. The rhythm may be atrial fibrillation with rapid ventricular response. There is frequent ventricular ectopy. Compared to the previous tracing of [**2200-4-23**] the rate has increased, apparently because of atrial fibrillation with rapid ventricular response and frequent ventricular ectopy. There is continued low limb lead voltage and delayed precordial R wave transition in the context of right ventricular conduction delay. Followup and clinical correlation are suggested. TRACING #1"
158116,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,159148,Normal sinus rhythm. Low voltage throughout the tracing in both the limb leads and the precordial leads. RSR' pattern in leads V1-V2. Left atrial abnormality. Poor R wave progression. Compared to the previous tracing of [**2122-10-27**] no diagnostic interval change.
107791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,159156,Sinus rhythm Normal ECG No previous tracing for comparison
185973,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,159162,Sinus bradycardia. Prolonged Q-T interval. Compared to the previous tracing the rate is slower. The Q-T interval is prolonged and the rhythm is now sinus. TRACING #3


In [0]:
hadm_top_50_icd_step_1_dc = spark.sql('''
select distinct 
    a.*,
    b.row_id,
    b.text
from top_50_icd_hadm_base a 
inner join noteevents2 b
on a.hadm_id = b.hadm_id
''')
hadm_top_50_icd_step_1_dc.registerTempTable("hadm_top_50_icd_step_1_dc_only")

In [0]:
import re
stopwords = []
def remstopwords(text):
        text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
        text = re.sub('<[^>]*>', '', text)
        text = re.sub('[\W]+', ' ', text.lower()) 
        text = re.sub(" \d+", " ", text)
        return " ".join([i for i in text.split() if i not in stopwords])

In [0]:
from pyspark.sql.functions import col
remstopwords_s = spark.udf.register("remstopwords", remstopwords)

hadm_top_50_icd_step_2 = hadm_top_50_icd_step_1.select(col("hadm_id"), col("row_id"), remstopwords_s(col("text")))
hadm_top_50_icd_step_2 = hadm_top_50_icd_step_2.withColumnRenamed("remstopwords(text)","clean_text")
hadm_top_50_icd_step_2.registerTempTable("hadm_top_50_icd_step_2")

In [0]:
from pyspark.sql.functions import col
remstopwords_s = spark.udf.register("remstopwords", remstopwords)

hadm_top_50_icd_step_2_dc = hadm_top_50_icd_step_1_dc.select((col("hadm_id")), col("row_id"), remstopwords_s(col("text")))
hadm_top_50_icd_step_2_dc = hadm_top_50_icd_step_2_dc.withColumnRenamed("remstopwords(text)","clean_text")
hadm_top_50_icd_step_2_dc.registerTempTable("hadm_top_50_icd_step_2_dc_only")

In [0]:
%sql
select * 
from hadm_top_50_icd_step_2_dc_only
limit 10

hadm_id row_id clean_text 189681 183 admission date discharge date date of birth sex f service neurosurgery allergies codeine attending chief complaint skull defect major surgical or invasive procedure s p cranioplasty on history of present illness yo female with a h o left frontal avm in the supplementary motor area the avm was treated with stereotactic radiosurgery gamma knife in in the patient developed a seizure disorder she developed headaches and after an mri and a digital angiogram showed no residual pathological vessels a contrast enhancing lesion with massive focal residual edema was diagnosed very likely represents radionecrosis the patient had midline shift and mass effect on she had a left craniotomy for resection of the radionecrosis she then presented to the office in with increased left facial swelling and incision drainage she was taken to the or for a wound washout and craniectomy she now returns for a cranioplasty after a long course of outpatient iv antibiotic therapy past medical history seizures h o radio therapy for avm has resid edema causing seizures dysrhythmia palps w panic attacks recent upper respiratory infection palpitations with panic attacks panic anxiety depression h o nephrolithiasis at yrs old tb as a child treated social history married lives with husband family history non contributory physical exam on admission aox3 perrl face symm tongue midline eom intact w o nystagmus speech clear and fluent comprehension intact follows commands no pronator mae upon discharge aox3 neuro intact mae incision c d i ambulating tolerating pos pertinent results ct head post op patient is status post left frontal cranioplasty persistent vasogenic edema in the left frontal lobe unchanged no shift of normally midline structures or acute hemorrhage identified pm wbc rbc hgb hct mcv mch mchc rdw pm plt count pm calcium phosphate magnesium pm estgfr using this pm glucose creat sodium potassium chloride total co2 anion gap brief hospital course yo female who was electively admitted for a cranioplasty with dr immediately post op she remained in the pacu overnight overnight she voided l and received l ns bolus pod she was transferred to the floor prior to transfer she was noted to have increase hr low bp and low urine output thus received l of ns on the floor she was oob to chair tolerating a regular diet she was neurologically intact and cleared for discharge on medications on admission fluticasone flonase prescribed by other provider mcg spray suspension sprays s each nostril daily as needed for nasal congestion levetiracetam keppra mg tablet tablet s by mouth twice a day no substitution levetiracetam keppra mg tablet tablet s by mouth at bedtime no substitution levetiracetam keppra mg tablet tablet s by mouth four times a day no substitution oseltamivir tamiflu mg capsule one capsule s by mouth twice a day x days venlafaxine mg tablet one tablet s by mouth twice a day acetaminophen tylenol otc dosage uncertain ibuprofen advil migraine otc mg capsule capsule s by mouth once a day as needed for headache discharge medications acetaminophen mg tablet sig tablets po q4h every hours as needed for pain t ha bisacodyl mg tablet delayed release e c sig two tablet delayed release e c po daily daily as needed for constipation docusate sodium mg capsule sig one capsule po bid times a day disp capsule s refills hydromorphone mg tablet sig tablets po q4h every hours as needed for pain disp tablet s refills venlafaxine mg tablet sig two tablet po bid times a day fluticasone mcg actuation spray suspension sig two spray nasal daily daily as needed for nasal congestion dexamethasone mg tablet sig one tablet po q6h every hours for days take mg q6hrs take mg q12 take mg q24 then stop disp tablet s refills levetiracetam mg tablet sig tablets po bid times a day discharge disposition home discharge diagnosis skull defect s p cranioplasty discharge condition neurologically stable discharge instructions general instructions have a friend f

In [0]:
hadm_top_50_icd_step_3 = spark.sql('''select distinct a.*
,b.clean_text
from hadm_top_50_icd_step_2 b
inner join hadm_top_50_icd_step_1 a 
on a.hadm_id = b.hadm_id
and a.row_id = b.row_id
''')
hadm_top_50_icd_step_3.registerTempTable("hadm_top_50_icd_step_3")

In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_3;
create table tempdb.hadm_top_50_icd_step_3 as (
select *
from hadm_top_50_icd_step_3
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select count(1)
,count(distinct hadm_id)
from tempdb.hadm_top_50_icd_step_3

count(1),count(DISTINCT hadm_id)
1817952,52726


In [0]:

%sql
select count(distinct case when (`4019` +  `2724` + `25000` + `4280` + `41401` + `53081` + `51881` + `42731` + `5849` + `5990`) >= 1 then hadm_id else null end)
,count(distinct hadm_id)
from tempdb.hadm_top_50_icd_step_3



count(DISTINCT CASE WHEN ((((((((((4019 + 2724) + 25000) + 4280) + 41401) + 53081) + 51881) + 42731) + 5849) + 5990) >= 1) THEN hadm_id ELSE NULL END),count(DISTINCT hadm_id)
40562,52726


In [0]:
(1817952 /2) 

Out[3]: 908976.0

In [0]:
454148 

In [0]:
hadm_top_50_icd_step_3_dc_only = spark.sql('''select distinct a.*
,b.clean_text
from hadm_top_50_icd_step_2_dc_only b
inner join hadm_top_50_icd_step_1_dc_only a 
on a.hadm_id = b.hadm_id
and a.row_id = b.row_id
''')
hadm_top_50_icd_step_3_dc_only.registerTempTable("hadm_top_50_icd_step_3_dc_only")

In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_3_dc_only;
create table tempdb.hadm_top_50_icd_step_3_dc_only as (
select *
from hadm_top_50_icd_step_3_dc_only
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select *
from tempdb.hadm_top_50_icd_step_3_dc_only
limit 10

hadm_id 0389 2449 25000 2720 2724 2760 2761 2762 2767 2851 2859 2875 3051 311 32723 4019 40390 40391 41071 412 41401 4168 4240 4241 42731 42789 4280 486 49390 496 5070 5119 5180 51881 53081 5845 5849 5859 5990 7742 78552 99592 9971 V053 V1582 V290 V4581 V4582 V5861 V5867 row_id text clean_text 129169 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 15888 Admission Date: [**2172-9-24**] Discharge Date: [**2172-9-29**]

Date of Birth: [**2113-12-12**] Sex: F

Service:

On the day of admission, her echocardiogram showed an
ejection fraction of 40%.

MEDICATIONS ON ADMISSION: Lipitor 60 mg q.d., Singulair 10
mg q.d., Vista 15 mg q.d., Vioxx 25 mg q.d., Glucophage 1000
mg p.o. b.i.d., Flovent, Albuterol p.r.n., ..................
q.d., Atenolol 50 mg q.d., Lisinopril 40 mg q.d., Avandia 6
mg, Vista 60 mg, ECASA 325 p.o. q.d.

LABORATORY DATA: Normal. Creatinine 0.8; white count 11.5,
hematocrit 35.7.

PHYSICAL EXAMINATION: Vital signs: She was afebrile.
Vitals signs stable. General: She was in no apparent
distress. HEENT: Pupils equal, round and reactive to light.
No jugular venous distention. No bruits. Neck: Supple.
Lungs: Clear to auscultation bilaterally. Heart: Regular,
rate and rhythm. Normal S1 and S2. No murmurs, rubs or
gallops. Abdomen: Soft, nontender, nondistended. Bowel
sounds present. Extremities: Warm and well perfused. No
varicosities. No clubbing, cyanosis,or edema. Pulses 2+
bilaterally, 1+ dorsalis pedis, no posterior tibial palpable.

HOSPITAL COURSE: She was prepped preoperatively for surgery.
She was taken to the Operating Room on [**2172-9-25**],
where coronary artery bypass grafting times four was
performed, and was transferred to the CSRU postoperatively.
She was weaned from her ventilator and extubated. She did
well. She was started on beta-blockers after her
Neo-Synephrine was weaned off, and her chest tubes were
removed.

Physical Therapy was consulted while in the Intensive Care
Unit to assess mobility and ambulation. It was felt that she
would be safe to go home at that time with continued therapy
while in the hospital. She continued to improve and was
started on Lopressor. She was hemodynamically stable and was
tolerating a clear diet. She was started on Lasix and
transferred to the floor. Postoperatively her Foley was
removed. On [**2172-9-28**], the patient improved on the
floor, and her wires were removed on postoperative day #4.
She had some slight drainage on [**2172-9-29**], and the
Steri-Strips were removed, and Betadine was placed on the
site, and the incision was watched. Tentatively the patient
is scheduled to be discharged home in stable condition on
[**2172-9-30**].

DISCHARGE MEDICATIONS: Avandia 6 mg p.o. q.d., Glucophage
100 mg p.o. b.i.d., Percocet [**1-15**] tab p.o. q.4 hours p.r.n.,
Motrin 400 mg p.o. q.4 hours p.r.n., ECASA 325 p.o. q.d.,
Colace 100 mg p.o. b.i.d., Zantac 150 mg p.o. b.i.d., KCl 20
mEq p.o. b.i.d., Lasix 20 mg p.o. b.i.d., Lopressor 25 mg
p.o. b.i.d.

CO[**Last Name (STitle) **]N ON DISCHARGE: The patient was discharged home in
stable condition and was instructed to follow-up with her
primary care physician [**Last Name (NamePattern4) **] [**1-15**] weeks and with Dr. [**Last Name (Prefixes) **]
in approximately four weeks.

DISCHARGE INSTRUCTIONS: The patient was instructed to watch
out signs of infection at the incision site.

DISCHARGE DIAGNOSIS:
1. Coronary artery disease status post coronary artery
bypass grafting times four.
2. Hypertension.
3. High cholesterol.
4. Diabetes.
5. Asthma.
6. Diverticulosis.

DISCHARGE STATUS: The patient was discharged home in stable
condition.






 [**Doctor Last Name 412**] [**Last Name (Prefixes) 413**], M.D. [**MD Number(1) 414**]

Dictated By:[**Last Name (NamePattern4) **]
MEDQUIST36

D: [**2172-9-29**] 17:10
T: [**2172-9-30**] 08:28
JOB#: [**Job Number 45620**]
 admission date discharge date date of birth sex f service on the day of admission her echocardiogram showed a

create dataset lookup table

In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_4;
create table tempdb.hadm_top_50_icd_step_4 as (
select * 
from (
select *
,'1' as group
from tempdb.hadm_top_50_icd_step_3
where rand(1234) < .5

union all

select *
,'2' as group
from tempdb.hadm_top_50_icd_step_3
--hadm_top_50_icd_step_3_dc_only
where rand(1234) >= .5

)a
)

num_affected_rows,num_inserted_rows


check to see if the splits matched the original dataset and that we have 0 duplicates

In [0]:
%sql
select count(1)
,row_id
from tempdb.hadm_top_50_icd_step_4
group by 2
having count(1) > 1

count(1),row_id


In [0]:
%sql
select count(distinct row_id)
,'splits' as type
from tempdb.hadm_top_50_icd_step_4

union all

select count(distinct row_id)
,'original set' as type
from tempdb.hadm_top_50_icd_step_3

count(DISTINCT row_id),type
1817952,splits
1817952,original set


further split second set to valid and test

In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_5;
create table tempdb.hadm_top_50_icd_step_5 as (
select * 
from (
      select *
      ,'train' as group_type
      from tempdb.hadm_top_50_icd_step_4
      where group = '1'

      union all
      
      select * 
      from (
            select *
            ,'test' as group_type
            from tempdb.hadm_top_50_icd_step_4
            where rand(1234) >= .5
            and group = '2'

            union all 

            select *
            ,'train' as group_type
            from tempdb.hadm_top_50_icd_step_4
            where rand(1234) < .5
            and group = '2'
            )a
      ) b
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
create table tempdb.hadm_dataset_lookup as (
select distinct 
    hadm_id,
    row_id,
    group_type
from tempdb.hadm_top_50_icd_step_5
);

check again to see if our splits matched original dataset and that we have 0 duplicates

In [0]:
%sql
select count(1)
,row_id
from tempdb.hadm_top_50_icd_step_5
group by 2
having count(1) > 1

count(1),row_id


In [0]:
%sql
select count(distinct row_id)
,'splits' as type
from tempdb.hadm_top_50_icd_step_5

union all

select count(distinct row_id)
,'original set' as type
from tempdb.hadm_top_50_icd_step_3

count(DISTINCT row_id),type
1817952,splits
1817952,original set


In [0]:
%sql
select * 
from tempdb.hadm_top_50_icd_step_5
limit 10

hadm_id,0389,2449,25000,2720,2724,2760,2761,2762,2767,2851,2859,2875,3051,311,32723,4019,40390,40391,41071,412,41401,4168,4240,4241,42731,42789,4280,486,49390,496,5070,5119,5180,51881,53081,5845,5849,5859,5990,7742,78552,99592,9971,V053,V1582,V290,V4581,V4582,V5861,V5867,row_id,text,clean_text,group,group_type
100020,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,360277,"58yom with terminal progressive MS since [**2119**] on baclofen abdominal pump. PMH of seizures only in the context of hyponatremia attributed to SIADH from paxil. Past 3 days pt having difficulty swallowing with bilateral hand/arm weakness, confusion and visual hallucinations, brought to OSH by wife1/8. Found to have Na 117 with +UTI and a witnessed 45second tonic clonic [**Year (4 digits) 2378**] that was treated with Ativan. Transferred to [**Hospital1 54**] CC6 [**11-30**] at [**2133**] SBP 196/100 received BP medications with minimal effect. At 2200 became unresponsive with SBP 70 received 800cc NS and placed NRB for o2 sat 81%.. Responded to NS with SBP 90/ and O2 weaned to NP. Arrived CCU(MICU boarder) awake , alert x3 with stable SBP 100. [**Year (4 digits) **], without status epilepticus Assessment: Last [**Year (4 digits) 2378**] at [**2163**] at OSH. Arrived slightly confused asking the same questions over however within 1 hr become less confused and alert and oriented x3. Unable to lower extremities and minimal movement bilateral upper extremities. Hands are weak , unable to open with hands in fists. Action: [**Year (4 digits) **] precautions, aspiration precautions, q2hr monitoring of MS while awake. Response: No seizures or hallucinations observed. Plan: Continue to monitor for further change in MS [**First Name (Titles) **] [**Last Name (Titles) 2378**] activity. Hypotension (not Shock) Assessment: Baseline BP 160-190 on many antihypertensive medications. SBP 90-100 with HR 52-68 without ectopy. Received total 1000cc of IVF with stable urine output. Action: Following SBP to maintain MAP>60 Response: SBP much lower with h/o HTN. Plan: Maintain fluid restrictions and monitor serial Na levels.",58yom with terminal progressive ms since on baclofen abdominal pump pmh of seizures only in the context of hyponatremia attributed to siadh from paxil past days pt having difficulty swallowing with bilateral hand arm weakness confusion and visual hallucinations brought to osh by wife1 found to have na with uti and a witnessed second tonic clonic that was treated with ativan transferred to cc6 at sbp received bp medications with minimal effect at became unresponsive with sbp received cc ns and placed nrb for o2 sat responded to ns with sbp and o2 weaned to np arrived ccu micu boarder awake alert x3 with stable sbp without status epilepticus assessment last at at osh arrived slightly confused asking the same questions over however within hr become less confused and alert and oriented x3 unable to lower extremities and minimal movement bilateral upper extremities hands are weak unable to open with hands in fists action precautions aspiration precautions q2hr monitoring of ms while awake response no seizures or hallucinations observed plan continue to monitor for further change in ms activity hypotension not shock assessment baseline bp on many antihypertensive medications sbp with hr without ectopy received total cc of ivf with stable urine output action following sbp to maintain map response sbp much lower with h o htn plan maintain fluid restrictions and monitor serial na levels,1,train
100021,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,974074,"[**2109-9-18**] 11:08 AM ANKLE (AP, MORTISE & LAT) RIGHT Clip # [**Clip Number (Radiology) 10391**] Reason: postop Admitting Diagnosis: BILATERAL ANKLE FX; NEW AFLUTTER ______________________________________________________________________________ [**Hospital 2**] MEDICAL CONDITION: 54 year old man s/p ORIF righ

repeat code with smaller dataset (discharge notes only)

In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_4_dc_only;
create table tempdb.hadm_top_50_icd_step_4_dc_only as (
select * 
from (
select *
,'1' as group
from tempdb.hadm_top_50_icd_step_3_dc_only
where rand(1234) < .5

union all

select *
,'2' as group
from tempdb.hadm_top_50_icd_step_3_dc_only
--hadm_top_50_icd_step_3_dc_only
where rand(1234) >= .5

)a
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
drop table if exists tempdb.hadm_top_50_icd_step_5_dc_only;
create table tempdb.hadm_top_50_icd_step_5_dc_only as (
select * 
from (
      select *
      ,'train' as group_type
      from tempdb.hadm_top_50_icd_step_4_dc_only
      where group = '1'

      union all
      
      select * 
      from (
            select *
            ,'test' as group_type
            from tempdb.hadm_top_50_icd_step_4_dc_only
            where rand(1234) >= .5
            and group = '2'

            union all 

            select *
            ,'train' as group_type
            from tempdb.hadm_top_50_icd_step_4_dc_only
            where rand(1234) < .5
            and group = '2'
            )a
      ) b
)

num_affected_rows,num_inserted_rows


stage 1 of the preprocessing is now complete, on to extracting features from word 2 vec and using a TF/IDF model

older code that I had issues with

In [0]:
# id_to_topicd9 , tru = get_id_to_topicd9("hadm_id", True, 50)

In [0]:
# stru_list = []
# stru_list.append(StructField("id", DoubleType(), True))
# for i in tru:
#   cur_stru = StructField(str(i), IntegerType(), True)
#   stru_list.append(cur_stru)
# stru_list.append(StructField("text", StringType(), True))
# tst_struct = StructType(stru_list)

sample output of dataset that is used later that I believe is causing an error

In [0]:
# id_type = 'hadm_id'
# topX = 50
# id_to_topicd9code, topicd9code = get_id_to_topicd9(id_type, True, topX)
# id_to_topicd9cat, topicd9cat = get_id_to_topicd9(id_type, False, topX)
# topX2 = topX
# topicd9 = topicd9code+topicd9cat
# mapper = dict(zip(topicd9, range(topX2)))
# #original code below, this is causing the issues
# id_to_topicd9 = id_to_topicd9code.fullOuterJoin(id_to_topicd9cat) \
# .map(lambda x: (x[0], \
#                                              (x[1][0] if x[1][0] else set()) | \
#                                              (x[1][1] if x[1][1] else set())))     # unpack manually

In [0]:


# id_to_topicd9_df = id_to_topicd9.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()

In [0]:
# import re

# def sparse2vec(mapper, data):
#     out = [0] * len(mapper)
#     if data != None:
#         for i in data:
#             out[mapper[i]] = 1
#     return out
    
# def get_id_to_texticd9(id_type, topX, stopwords=[]):
#     def remstopwords(text):
#         text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
#         text = re.sub('<[^>]*>', '', text)
#         text = re.sub('[\W]+', ' ', text.lower()) 
#         text = re.sub(" \d+", " ", text)
#         return " ".join([i for i in text.split() if i not in stopwords])
    
#     id_to_topicd9code, topicd9code = get_id_to_topicd9(id_type, True, topX)
#     id_to_topicd9cat, topicd9cat = get_id_to_topicd9(id_type, False, topX)
#     topX2 = topX
#     topicd9 = topicd9code+topicd9cat
#     mapper = dict(zip(topicd9, range(topX2)))
#     #original code below, this is causing the issues
#     id_to_topicd9 = id_to_topicd9code.fullOuterJoin(id_to_topicd9cat) \
#     .map(lambda x: (x[0], \
#                                                  (x[1][0] if x[1][0] else set()) | \
#                                                  (x[1][1] if x[1][1] else set())))     # unpack manually
    
    
# #         .map(lambda (id_, (icd9code, icd9cat)): (id_, \
# #                                                  (icd9code if icd9code else set()) | \
# #                                                  (icd9cat if icd9cat else set())))
# # here are a few attempts at converting this code below, above is the original code
# #         .map(lambda t : (t[0],t[1],t[2]) if len(t) > 2 else ((t[0],t[1],set()) if len(t) > 1  else (t[0],set(),set())))
# #              lambda x : x*2 if x < 10 else (x*3 if x < 20 else x)
# #     lambda <args> : <return Value> if <condition > ( <return value > if <condition> else <return value>)
# #           .map(lambda t: (t[0] if len(t) > 0 else set() , t[1] if len(t) > 1 else set() ,t[2] if len(t) > 2  else set()))
# #         .map(lambda t: t[0], t[1] if t[1] else set() ,t[2] if t[2] else set())
        
#     ne_topX = spar_df.rdd \
#         .filter(lambda x: x.CATEGORY == "Discharge summary") \
#         .map(lambda x: (x.HADM_ID if id_type=="hadm_id" else x.SUBJECT_ID, x.TEXT)) \
#         .groupByKey() \
#         .mapValues(lambda x: " ".join(x)) \
#         .leftOuterJoin(id_to_topicd9) \
#         .map(lambda x: \
#              [x[0]]+sparse2vec(mapper, x[2] if len(x) > 2 else [])+[x[1] if len(x) > 1 else [] if len(stopwords) == 0 else remstopwords(x[1] if len(x) > 1 else [])])
# #              list(Vectors.sparse(topX, dict.fromkeys(map(lambda x: mapper[x], icd9), 1))))
            
#     return spark.createDataFrame(ne_topX, tst_struct), topicd9

In [0]:
# id_type = ''
# topx
# id_to_topicd9code, topicd9code = get_id_to_topicd9(id_type, True, topX)
# id_to_topicd9cat, topicd9cat = get_id_to_topicd9(id_type, False, topX)
# topX2 = 2 * topX
# topicd9 = topicd9code+topicd9cat
# mapper = dict(zip(topicd9, range(topX2)))
# #original code below, this is causing the issues
# id_to_topicd9 = id_to_topicd9code.fullOuterJoin(id_to_topicd9cat) \
# .map(lambda x: (x[0], \
#                                              (x[1][0] if x[1][0] else set()) | \
#                                              (x[1][1] if x[1][1] else set())))     # unpack manually

In [0]:
# spark.createDataFrame(id_to_topicd9)

In [0]:
# %sql
# SELECT  distinct icd9_code AS icd9_code, COUNT(DISTINCT hadm_id) AS score
# FROM diagnoses_icd_m2
# GROUP BY icd9_code
# order by 2 desc
# limit 50

In [0]:
# import pickle

# ICD9CODES = spark.sql("""
# SELECT DISTINCT icd9_code FROM diagnoses_icd_m2
# """).rdd.map(lambda x: x.icd9_code).collect()
# ICD9CODES = [str(i).lower() for i in ICD9CODES]

In [0]:
# import time
# t0 = time.time()

# df_id2texticd9, topicd9 = get_id_to_texticd9("hadm_id", 50)


In [0]:
# df_id2texticd9.registerTempTable("df_id2texticd9")

descriptive stats on datasets

In [0]:
spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), COUNT(DISTINCT hadm_id)
FROM noteevents
""").show()
spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), COUNT(DISTINCT hadm_id)
FROM noteevents2
""").show()

+--------+--------------------------+-----------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|
+--------+--------------------------+-----------------------+
 2083180| 46146| 58361|
+--------+--------------------------+-----------------------+

+--------+--------------------------+-----------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|
+--------+--------------------------+-----------------------+
 59652| 41127| 52726|
+--------+--------------------------+-----------------------+

In [0]:
spark.sql("""
SELECT COUNT(DISTINCT hadm_id) AS hadm_count
FROM diagnoses_icd_m2
WHERE icd9_code IN
    (SELECT icd9_code
    FROM diagnoses_icd_m2
    GROUP BY icd9_code
    ORDER BY COUNT(DISTINCT hadm_id) DESC
    LIMIT 10)
""").show()

spark.sql("""
SELECT COUNT(DISTINCT hadm_id) AS hadm_count
FROM diagnoses_icd_m2
WHERE icd9_code IN
    (SELECT icd9_code
    FROM diagnoses_icd_m2
    GROUP BY icd9_code
    ORDER BY COUNT(DISTINCT hadm_id) DESC
    LIMIT 50)
""").show()

spark.sql("""
SELECT COUNT(DISTINCT hadm_id) AS hadm_count
FROM diagnoses_icd_m2
WHERE icd9_code IN
    (SELECT icd9_code
    FROM diagnoses_icd_m2
    GROUP BY icd9_code
    ORDER BY COUNT(DISTINCT hadm_id) DESC
    LIMIT 100)
""").show()

+----------+
hadm_count|
+----------+
 40562|
+----------+

+----------+
hadm_count|
+----------+
 49354|
+----------+

+----------+
hadm_count|
+----------+
 50625|
+----------+

Categories:

In [0]:
spark.sql("""
SELECT DISTINCT(category)
FROM noteevents
""").show(truncate=False)

+-----------------+
category |
+-----------------+
ECG |
Echo |
Discharge summary|
Respiratory |
Nursing |
General |
Consult |
Nutrition |
Physician |
Pharmacy |
Rehab Services |
Case Management |
Social Work |
Radiology |
Nursing/other |
+-----------------+

<h4> diagnoses_icd: many (icd_code) to one (hadm_id) <br>
<h5> Basic Counts:

In [0]:
spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT ICD9_CODE)
FROM diagnoses_icd_m
""").show()

spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT LOWER(ICD9_CODE))
FROM diagnoses_icd_m
""").show()

+--------+--------------------------+-----------------------+-------------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT ICD9_CODE)|
+--------+--------------------------+-----------------------+-------------------------+
 651047| 46520| 58976| 6984|
+--------+--------------------------+-----------------------+-------------------------+

+--------+--------------------------+-----------------------+--------------------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT lower(ICD9_CODE))|
+--------+--------------------------+-----------------------+--------------------------------+
 651047| 46520| 58976| 6984|
+--------+--------------------------+-----------------------+--------------------------------+

<h4> diagnoses_icd: one (icd_code) to one (hadm_id) <br>
<h5> Basic Counts:

In [0]:
spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT ICD9_CODE)
FROM diagnoses_icd_o
""").show()

spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT LOWER(ICD9_CODE))
FROM diagnoses_icd_o
""").show()

+--------+--------------------------+-----------------------+-------------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT ICD9_CODE)|
+--------+--------------------------+-----------------------+-------------------------+
 46499| 46499| 46499| 2564|
+--------+--------------------------+-----------------------+-------------------------+

+--------+--------------------------+-----------------------+--------------------------------+
count(1)|count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT lower(ICD9_CODE))|
+--------+--------------------------+-----------------------+--------------------------------+
 46499| 46499| 46499| 2564|
+--------+--------------------------+-----------------------+--------------------------------+

Just to check if I really did get "seq_num = 1" for all diagnosis, the code below should return empty.

In [0]:

# check code
spark.sql("""
SELECT *
FROM diagnoses_icd_o
WHERE seq_num <> 1
""").show()

+------+----------+-------+-------+---------+
row_id|subject_id|hadm_id|seq_num|icd9_code|
+------+----------+-------+-------+---------+
+------+----------+-------+-------+---------+

<h4> noteevents and diagnoses_icd (one to one) <br>
<h5> Basic Counts:

In [0]:
spark.sql("""
SELECT COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT icd9_code)
FROM diagnoses_icd_o2
""").show()

+--------------------------+-----------------------+-------------------------+
count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT icd9_code)|
+--------------------------+-----------------------+-------------------------+
 40822| 40822| 2485|
+--------------------------+-----------------------+-------------------------+

Top 50 ICD 9 codes based on "subject_id" count

In [0]:
spark.sql("""
SELECT a.icd9_code, sid_count, SHORT_TITLE, LONG_TITLE FROM
    (SELECT icd9_code, COUNT(DISTINCT subject_id) AS sid_count
    FROM diagnoses_icd_o2
    GROUP BY icd9_code
    ORDER BY sid_count DESC
    LIMIT 50) AS a
JOIN diagnoses_icd_desc
ON diagnoses_icd_desc.ICD9_CODE = a.icd9_code
order by sid_count desc
""").show(n=50, truncate=50)

+---------+---------+------------------------+--------------------------------------------------+
icd9_code|sid_count| SHORT_TITLE| LONG_TITLE|
+---------+---------+------------------------+--------------------------------------------------+
 41401| 3097|Crnry athrscl natve vssl|Coronary atherosclerosis of native coronary artery|
 41071| 1411|Subendo infarct, initial|Subendocardial infarction, initial episode of care|
 0389| 1375| Septicemia NOS| Unspecified septicemia|
 V3001| 1371| Single lb in-hosp w cs|Single liveborn, born in hospital, delivered by...|
 4241| 983| Aortic valve disorder| Aortic valve disorders|
 V3000| 940|Single lb in-hosp w/o cs|Single liveborn, born in hospital, delivered wi...|
 431| 871|Intracerebral hemorrhage| Intracerebral hemorrhage|
 V3101| 832|Twin-mate lb-in hos w cs|Twin birth, mate liveborn, born in hospital, de...|
 51881| 718|Acute respiratry failure| Acute respiratory failure|
 4240| 482| Mitral valve disorder| Mitral valve disorders|
 430| 437| Subarachnoid hemorrhage| Subarachnoid hemorrhage|
 486| 433| Pneumonia, organism NOS| Pneumonia, organism unspecified|
 5070| 429| Food/vomit pneumonitis| Pneumonitis due to inhalation of food or vomitus|
 41011| 418| AMI anterior wall, init|Acute myocardial infarction of other anterior w...|
 41041| 418| AMI inferior wall, init|Acute myocardial infarction of other inferior w...|
 4280| 367| CHF NOS| Congestive heart failure, unspecified|
 5849| 336|Acute kidney failure NOS| Acute kidney failure, unspecified|
 43411| 286| Crbl emblsm w infrct| Cerebral embolism with cerebral infarction|
 5770| 284| Acute pancreatitis| Acute pancreatitis|
 43491| 277|Crbl art ocl NOS w infrc|Cerebral artery occlusion, unspecified with cer...|
 5789| 272| Gastrointest hemorr NOS| Hemorrhage of gastrointestinal tract, unspecified|
 41519| 268| Pulm embol/infarct NEC| Other pulmonary embolism and infarction|
 1983| 266| Sec mal neo brain/spine|Secondary malignant neoplasm of brain and spina...|
 85221| 243| Subdural hem w/o coma|Subdural hemorrhage following injury without me...|
 42731| 237| Atrial fibrillation| Atrial fibrillation|
 V3401| 230|Oth mult lb-in hosp w cs|Other multiple birth (three or more), mates all...|
 03842| 226| E coli septicemia| Septicemia due to escherichia coli [E. coli]|
 99859| 201| Other postop infection| Other postoperative infection|
 56212| 194| Dvrtclo colon w hmrhg| Diverticulosis of colon with hemorrhage|
 4321| 180| Subdural hemorrhage| Subdural hemorrhage|
 85220| 172| Traumatic subdural hem|Subdural hemorrhage following injury without me...|
 4271| 169| Parox ventric tachycard| Paroxysmal ventricular tachycardia|
 5712| 167| Alcohol cirrhosis liver| Alcoholic cirrhosis of liver|
 42823| 163| Ac on chr syst hrt fail| Acute on chronic systolic heart failure|
 99811| 160| Hemorrhage complic proc| Hemorrhage complicating a procedure|
 4373| 159|Nonrupt cerebral aneurym| Cerebral aneurysm, nonruptured|
 9654| 148|Pois-arom analgesics NEC|Poisoning by aromatic analgesics, not elsewhere...|
 42833| 145|Ac on chr diast hrt fail| Acute on chronic diastolic heart failure|
 4412| 144|Thoracic aortic aneurysm| Thoracic aneurysm without mention of rupture|
 49121| 144|Obs chr bronc w(ac) exac|Obstructive chronic bronchitis with (acute) exa...|
 042| 142| Human immuno virus dis| Human immunodeficiency virus [HIV] disease|
 03849| 140| Gram-neg septicemia NEC| Other septicemia due to gram-negative organisms|
 51884| 140|Acute & chronc resp fail| Acute and chronic respiratory failure|
 44101| 139| Dsct of thoracic aorta| Dissection of aorta, thoracic|
 4414| 138| Abdom aortic aneurysm| Abdominal aneurysm without mention of rupture|
 43310| 138| Ocl crtd art wo infrct|Occlusion and stenosis of carotid artery withou...|
 29181| 137| Alcohol withdrawal| Alcohol withdrawal|
 3962| 137|Mitral insuf/aort stenos|Mitral valve insufficiency and aortic valve ste...|
 1623| 136| Mal neo upper lobe lung|Malignant neoplasm of upper lobe, bronchus or lung|
 250

Top 50 ICD 9 codes based on "hadm_id" count

In [0]:
spark.sql("""
SELECT a.icd9_code, hadm_count, SHORT_TITLE, LONG_TITLE FROM
    (SELECT icd9_code, COUNT(DISTINCT hadm_id) AS hadm_count
    FROM diagnoses_icd_o2
    GROUP BY icd9_code
    ORDER BY hadm_count DESC
    LIMIT 50) AS a
JOIN diagnoses_icd_desc
ON diagnoses_icd_desc.ICD9_CODE = a.icd9_code
ORDER BY hadm_count DESC
""").show(n=50, truncate=50)

+---------+----------+------------------------+--------------------------------------------------+
icd9_code|hadm_count| SHORT_TITLE| LONG_TITLE|
+---------+----------+------------------------+--------------------------------------------------+
 41401| 3097|Crnry athrscl natve vssl|Coronary atherosclerosis of native coronary artery|
 41071| 1411|Subendo infarct, initial|Subendocardial infarction, initial episode of care|
 0389| 1375| Septicemia NOS| Unspecified septicemia|
 V3001| 1371| Single lb in-hosp w cs|Single liveborn, born in hospital, delivered by...|
 4241| 983| Aortic valve disorder| Aortic valve disorders|
 V3000| 940|Single lb in-hosp w/o cs|Single liveborn, born in hospital, delivered wi...|
 431| 871|Intracerebral hemorrhage| Intracerebral hemorrhage|
 V3101| 832|Twin-mate lb-in hos w cs|Twin birth, mate liveborn, born in hospital, de...|
 51881| 718|Acute respiratry failure| Acute respiratory failure|
 4240| 482| Mitral valve disorder| Mitral valve disorders|
 430| 437| Subarachnoid hemorrhage| Subarachnoid hemorrhage|
 486| 433| Pneumonia, organism NOS| Pneumonia, organism unspecified|
 5070| 429| Food/vomit pneumonitis| Pneumonitis due to inhalation of food or vomitus|
 41011| 418| AMI anterior wall, init|Acute myocardial infarction of other anterior w...|
 41041| 418| AMI inferior wall, init|Acute myocardial infarction of other inferior w...|
 4280| 367| CHF NOS| Congestive heart failure, unspecified|
 5849| 336|Acute kidney failure NOS| Acute kidney failure, unspecified|
 43411| 286| Crbl emblsm w infrct| Cerebral embolism with cerebral infarction|
 5770| 284| Acute pancreatitis| Acute pancreatitis|
 43491| 277|Crbl art ocl NOS w infrc|Cerebral artery occlusion, unspecified with cer...|
 5789| 272| Gastrointest hemorr NOS| Hemorrhage of gastrointestinal tract, unspecified|
 41519| 268| Pulm embol/infarct NEC| Other pulmonary embolism and infarction|
 1983| 266| Sec mal neo brain/spine|Secondary malignant neoplasm of brain and spina...|
 85221| 243| Subdural hem w/o coma|Subdural hemorrhage following injury without me...|
 42731| 237| Atrial fibrillation| Atrial fibrillation|
 V3401| 230|Oth mult lb-in hosp w cs|Other multiple birth (three or more), mates all...|
 03842| 226| E coli septicemia| Septicemia due to escherichia coli [E. coli]|
 99859| 201| Other postop infection| Other postoperative infection|
 56212| 194| Dvrtclo colon w hmrhg| Diverticulosis of colon with hemorrhage|
 4321| 180| Subdural hemorrhage| Subdural hemorrhage|
 85220| 172| Traumatic subdural hem|Subdural hemorrhage following injury without me...|
 4271| 169| Parox ventric tachycard| Paroxysmal ventricular tachycardia|
 5712| 167| Alcohol cirrhosis liver| Alcoholic cirrhosis of liver|
 42823| 163| Ac on chr syst hrt fail| Acute on chronic systolic heart failure|
 99811| 160| Hemorrhage complic proc| Hemorrhage complicating a procedure|
 4373| 159|Nonrupt cerebral aneurym| Cerebral aneurysm, nonruptured|
 9654| 148|Pois-arom analgesics NEC|Poisoning by aromatic analgesics, not elsewhere...|
 42833| 145|Ac on chr diast hrt fail| Acute on chronic diastolic heart failure|
 4412| 144|Thoracic aortic aneurysm| Thoracic aneurysm without mention of rupture|
 49121| 144|Obs chr bronc w(ac) exac|Obstructive chronic bronchitis with (acute) exa...|
 042| 142| Human immuno virus dis| Human immunodeficiency virus [HIV] disease|
 03849| 140| Gram-neg septicemia NEC| Other septicemia due to gram-negative organisms|
 51884| 140|Acute & chronc resp fail| Acute and chronic respiratory failure|
 44101| 139| Dsct of thoracic aorta| Dissection of aorta, thoracic|
 4414| 138| Abdom aortic aneurysm| Abdominal aneurysm without mention of rupture|
 43310| 138| Ocl crtd art wo infrct|Occlusion and stenosis of carotid artery withou...|
 29181| 137| Alcohol withdrawal| Alcohol withdrawal|
 3962| 137|Mitral insuf/aort stenos|Mitral valve insufficiency and aortic valve ste...|
 1623| 136| Mal neo upper lobe lung|Malignant neoplasm of upper lobe, bronchus or lung|
 

<h4> noteevents and diagnoses_icd (many to one) <br>
<h5> Basic Counts:

In [0]:
spark.sql("""
SELECT COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT icd9_code)
FROM diagnoses_icd_m2
""").show()

spark.sql("""
SELECT COUNT(DISTINCT subject_id), 
COUNT(DISTINCT hadm_id), COUNT(DISTINCT icd9_code)
FROM (
    SELECT row_id, subject_id, diagnoses_icd_m.hadm_id AS hadm_id,
    seq_num, icd9_code
    FROM diagnoses_icd_m JOIN (SELECT DISTINCT hadm_id FROM noteevents) AS a
    ON diagnoses_icd_m.hadm_id = a.hadm_id
)
""").show()

+--------------------------+-----------------------+-------------------------+
count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT icd9_code)|
+--------------------------+-----------------------+-------------------------+
 41127| 52726| 6918|
+--------------------------+-----------------------+-------------------------+

+--------------------------+-----------------------+-------------------------+
count(DISTINCT subject_id)|count(DISTINCT hadm_id)|count(DISTINCT icd9_code)|
+--------------------------+-----------------------+-------------------------+
 46139| 58361| 6967|
+--------------------------+-----------------------+-------------------------+

Top ICD 9 codes based on "subject_id" count

In [0]:
spark.sql("""
SELECT a.icd9_code, sid_count, SHORT_TITLE, LONG_TITLE FROM
    (SELECT icd9_code, COUNT(DISTINCT subject_id) AS sid_count
    FROM diagnoses_icd_m2
    GROUP BY icd9_code
    ORDER BY sid_count DESC
    LIMIT 50) AS a
JOIN diagnoses_icd_desc
ON diagnoses_icd_desc.ICD9_CODE = a.icd9_code
ORDER BY sid_count DESC
""").show(n=50, truncate=50)

+---------+---------+------------------------+--------------------------------------------------+
icd9_code|sid_count| SHORT_TITLE| LONG_TITLE|
+---------+---------+------------------------+--------------------------------------------------+
 4019| 17138| Hypertension NOS| Unspecified essential hypertension|
 41401| 10579|Crnry athrscl natve vssl|Coronary atherosclerosis of native coronary artery|
 42731| 10053| Atrial fibrillation| Atrial fibrillation|
 4280| 9669| CHF NOS| Congestive heart failure, unspecified|
 5849| 7505|Acute kidney failure NOS| Acute kidney failure, unspecified|
 2724| 7324| Hyperlipidemia NEC/NOS| Other and unspecified hyperlipidemia|
 25000| 7181|DMII wo cmp nt st uncntr|Diabetes mellitus without mention of complicati...|
 51881| 6493|Acute respiratry failure| Acute respiratory failure|
 5990| 5687|Urin tract infection NOS| Urinary tract infection, site not specified|
 2720| 5199| Pure hypercholesterolem| Pure hypercholesterolemia|
 53081| 5148| Esophageal reflux| Esophageal reflux|
 2859| 4895| Anemia NOS| Anemia, unspecified|
 486| 4329| Pneumonia, organism NOS| Pneumonia, organism unspecified|
 2851| 4195| Ac posthemorrhag anemia| Acute posthemorrhagic anemia|
 2762| 4021| Acidosis| Acidosis|
 2449| 3732| Hypothyroidism NOS| Unspecified acquired hypothyroidism|
 496| 3491| Chr airway obstruct NEC|Chronic airway obstruction, not elsewhere class...|
 99592| 3449| Severe sepsis| Severe sepsis|
 5070| 3319| Food/vomit pneumonitis| Pneumonitis due to inhalation of food or vomitus|
 0389| 3304| Septicemia NOS| Unspecified septicemia|
 V5861| 3126|Long-term use anticoagul| Long-term (current) use of anticoagulants|
 3051| 2926| Tobacco use disorder| Tobacco use disorder|
 41071| 2861|Subendo infarct, initial|Subendocardial infarction, initial episode of care|
 311| 2859| Depressive disorder NEC| Depressive disorder, not elsewhere classified|
 5859| 2855| Chronic kidney dis NOS| Chronic kidney disease, unspecified|
 40390| 2784|Hy kid NOS w cr kid I-IV|Hypertensive chronic kidney disease, unspecifie...|
 2761| 2757| Hyposmolality| Hyposmolality and/or hyponatremia|
 2875| 2751| Thrombocytopenia NOS| Thrombocytopenia, unspecified|
 412| 2723| Old myocardial infarct| Old myocardial infarction|
 4240| 2613| Mitral valve disorder| Mitral valve disorders|
 5119| 2528| Pleural effusion NOS| Unspecified pleural effusion|
 V290| 2519| NB obsrv suspct infect| Observation for suspected infectious condition|
 V1582| 2486| History of tobacco use| Personal history of tobacco use|
 78552| 2335| Septic shock| Septic shock|
 4241| 2285| Aortic valve disorder| Aortic valve disorders|
 9971| 2278| Surg compl-heart| Cardiac complications, not elsewhere classified|
 42789| 2259|Cardiac dysrhythmias NEC| Other specified cardiac dysrhythmias|
 V4581| 2256| Aortocoronary bypass| Aortocoronary bypass status|
 V4582| 2202| Status-post ptca|Percutaneous transluminal coronary angioplasty ...|
 7742| 2174|Neonat jaund preterm del|Neonatal jaundice associated with preterm delivery|
 5845| 2122|Ac kidny fail, tubr necr|Acute kidney failure with lesion of tubular nec...|
 V053| 2116|Need prphyl vc vrl hepat|Need for prophylactic vaccination and inoculati...|
 5180| 2055| Pulmonary collapse| Pulmonary collapse|
 2760| 2054| Hyperosmolality| Hyperosmolality and/or hypernatremia|
 45829| 2035|Iatrogenc hypotnsion NEC| Other iatrogenic hypotension|
 V5867| 1987|Long-term use of insulin| Long-term (current) use of insulin|
 2767| 1877| Hyperpotassemia| Hyperpotassemia|
 4589| 1862| Hypotension NOS| Hypotension, unspecified|
 4168| 1780|Chr pulmon heart dis NEC| Other chronic pulmonary heart diseases|
+---------+---------+------------------------+--------------------------------------------------+

In [0]:
tmp = spark.sql("""
SELECT a.icd9_code, hadm_count, SHORT_TITLE, LONG_TITLE FROM
    (SELECT icd9_code, COUNT(DISTINCT hadm_id) AS hadm_count
    FROM diagnoses_icd_m2
    GROUP BY icd9_code
    ORDER BY hadm_count DESC) AS a
JOIN diagnoses_icd_desc
ON diagnoses_icd_desc.ICD9_CODE = a.icd9_code
order by 2 desc
""")

tmp.show(n=50, truncate=50)

tmp2 = tmp.toPandas()

+---------+----------+------------------------+--------------------------------------------------+
icd9_code|hadm_count| SHORT_TITLE| LONG_TITLE|
+---------+----------+------------------------+--------------------------------------------------+
 4019| 20046| Hypertension NOS| Unspecified essential hypertension|
 4280| 12842| CHF NOS| Congestive heart failure, unspecified|
 42731| 12589| Atrial fibrillation| Atrial fibrillation|
 41401| 12178|Crnry athrscl natve vssl|Coronary atherosclerosis of native coronary artery|
 5849| 8906|Acute kidney failure NOS| Acute kidney failure, unspecified|
 25000| 8783|DMII wo cmp nt st uncntr|Diabetes mellitus without mention of complicati...|
 2724| 8503| Hyperlipidemia NEC/NOS| Other and unspecified hyperlipidemia|
 51881| 7249|Acute respiratry failure| Acute respiratory failure|
 5990| 6442|Urin tract infection NOS| Urinary tract infection, site not specified|
 53081| 6154| Esophageal reflux| Esophageal reflux|
 2720| 5766| Pure hypercholesterolem| Pure hypercholesterolemia|
 2859| 5295| Anemia NOS| Anemia, unspecified|
 2449| 4785| Hypothyroidism NOS| Unspecified acquired hypothyroidism|
 486| 4732| Pneumonia, organism NOS| Pneumonia, organism unspecified|
 2851| 4499| Ac posthemorrhag anemia| Acute posthemorrhagic anemia|
 2762| 4358| Acidosis| Acidosis|
 496| 4296| Chr airway obstruct NEC|Chronic airway obstruction, not elsewhere class...|
 99592| 3792| Severe sepsis| Severe sepsis|
 V5861| 3697|Long-term use anticoagul| Long-term (current) use of anticoagulants|
 5070| 3592| Food/vomit pneumonitis| Pneumonitis due to inhalation of food or vomitus|
 0389| 3580| Septicemia NOS| Unspecified septicemia|
 5859| 3367| Chronic kidney dis NOS| Chronic kidney disease, unspecified|
 40390| 3350|Hy kid NOS w cr kid I-IV|Hypertensive chronic kidney disease, unspecifie...|
 311| 3347| Depressive disorder NEC| Depressive disorder, not elsewhere classified|
 3051| 3272| Tobacco use disorder| Tobacco use disorder|
 412| 3203| Old myocardial infarct| Old myocardial infarction|
 2875| 3002| Thrombocytopenia NOS| Thrombocytopenia, unspecified|
 41071| 3001|Subendo infarct, initial|Subendocardial infarction, initial episode of care|
 2761| 2985| Hyposmolality| Hyposmolality and/or hyponatremia|
 V4581| 2943| Aortocoronary bypass| Aortocoronary bypass status|
 4240| 2876| Mitral valve disorder| Mitral valve disorders|
 V1582| 2741| History of tobacco use| Personal history of tobacco use|
 5119| 2693| Pleural effusion NOS| Unspecified pleural effusion|
 V4582| 2651| Status-post ptca|Percutaneous transluminal coronary angioplasty ...|
 40391| 2566| Hyp kid NOS w cr kid V|Hypertensive chronic kidney disease, unspecifie...|
 V290| 2529| NB obsrv suspct infect| Observation for suspected infectious condition|
 4241| 2517| Aortic valve disorder| Aortic valve disorders|
 78552| 2501| Septic shock| Septic shock|
 V5867| 2497|Long-term use of insulin| Long-term (current) use of insulin|
 42789| 2396|Cardiac dysrhythmias NEC| Other specified cardiac dysrhythmias|
 32723| 2328| Obstructive sleep apnea| Obstructive sleep apnea (adult)(pediatric)|
 9971| 2313| Surg compl-heart| Cardiac complications, not elsewhere classified|
 5845| 2223|Ac kidny fail, tubr necr|Acute kidney failure with lesion of tubular nec...|
 2760| 2221| Hyperosmolality| Hyperosmolality and/or hypernatremia|
 7742| 2183|Neonat jaund preterm del|Neonatal jaundice associated with preterm delivery|
 5180| 2137| Pulmonary collapse| Pulmonary collapse|
 V053| 2119|Need prphyl vc vrl hepat|Need for prophylactic vaccination and inoculati...|
 4168| 2118|Chr pulmon heart dis NEC| Other chronic pulmonary heart diseases|
 49390| 2113| Asthma NOS| Asthma, unspecified type, unspecified|
 2767| 2111| Hyperpotassemia| Hyperpotassemia|
+---------+----------+------------------------+--------------------------------------------------+
only showing top 50 rows